### 🛠️ Tools(도구)

AI 애플리케이션이 외부 시스템(API, 데이터베이스, 파일 시스템 등)과 상호 작용할 수 있도록 모델의 기능을 확장하는 구성 요소입니다.

#### 주요 개념 및 기능

+ 기능 확장: Tools는 모델이 구조화된 입력을 사용하여 외부 세계와 상호 작용할 수 있게 합니다.
+ 구조: Tools는 호출 가능한 함수와 해당 입력 스키마를 캡슐화합니다.
+ 도구 호출 (Tool Calling): 호환되는 채팅 모델에 Tools를 전달하면, 모델은 Tools를 호출할지 여부와 사용할 인수를 결정할 수 있습니다. 이 시나리오에서 도구 호출은 모델이 지정된 입력 스키마를 따르는 요청을 생성하도록 합니다.
+ 서버 측 도구 사용: 일부 채팅 모델(예: OpenAI, Anthropic, Gemini)은 웹 검색이나 코드 인터프리터와 같은 내장된 도구를 제공하며, 이는 서버 측에서 실행됩니다.

#### Tools 생성 및 정의

Tools를 생성하는 가장 기본적인 방법은 @tool 데코레이터를 사용하는 것입니다.

+ 기본 정의: 함수의 **docstring**은 모델이 Tools 사용 시점을 이해하는 데 도움을 주는 도구 설명이 됩니다.
+ 필수 사항: 도구의 입력 스키마를 정의하기 위해 **Type hints**가 필수입니다. 독스트링은 도구의 목적을 명확히 하는 데 도움이 되도록 유익하고 간결해야 합니다.

In [2]:
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """고객 데이터베이스에서 쿼리와 일치하는 레코드를 검색합니다.

    Args:
        query: 찾고자 하는 검색어
        limit: 반환할 최대 결과 개수
    """
    return f"'{query}'에 대한 {limit}개의 결과를 찾았습니다"

In [3]:
search_database

StructuredTool(name='search_database', description='고객 데이터베이스에서 쿼리와 일치하는 레코드를 검색합니다.\n\nArgs:\n    query: 찾고자 하는 검색어\n    limit: 반환할 최대 결과 개수', args_schema=<class 'langchain_core.utils.pydantic.search_database'>, func=<function search_database at 0x11073cc20>)

In [4]:
from langchain.tools import tool

@tool("web_search")  # 사용자 정의 이름
def search(query: str) -> str:
    """웹에서 정보를 검색합니다."""
    return f"검색 결과: {query}"

print(search.name)  # web_search

web_search


In [ ]:
from langchain.tools import tool

# description는 독스트링보다 높은 우선순위로 Tool 사용의도를 모델에게 전달합니다(독스트링을 보지 않고도 사용 여부 결정).
@tool("calculator", description="산술 계산을 수행합니다. 모든 수학 문제에 이 도구를 사용하십시오.")
def calc(expression: str) -> str:
    """수학적 표현식을 평가합니다."""
    # eval() 함수는 보안상의 위험이 있으므로 실제 운영 환경에서는 사용에 주의해야 합니다.
    return str(eval(expression))

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.tools import tool

# Pydantic 스키마는 LLM이 도구의 인수를 이해하는 데 도움을 줍니다.
class WeatherInput(BaseModel):
    """날씨 쿼리를 위한 입력입니다."""
    # Field의 description은 모델이 인수를 이해하는 데 도움을 줍니다.
    location: str = Field(description="도시 이름 또는 좌표")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="온도 단위 선호도"
    )
    include_forecast: bool = Field(
        default=False,
        description="5일 일기 예보 포함 여부"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """현재 날씨와 선택적 예보를 가져옵니다."""
    temp = 22 if units == "celsius" else 72
    # 반환 문자열 번역
    result = f"{location}의 현재 날씨: {temp} {units[0].upper()}도"
    if include_forecast:
        result += "\n향후 5일: 맑음"
    return result